In [1]:
import tweepy
import configparser
import os
import pandas as pd
import time

In [10]:
# read all credentials from config file
config = configparser.ConfigParser()
config.read('config.ini')
print('successfully read config file')

api_key             = config['twitter']['api_key']
api_secret          = config['twitter']['api_secret']

access_token        = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']

bearer_token        = r"AAAAAAAAAAAAAAAAAAAAADL6lAEAAAAAyjVNm%2FOa6z1s2IKm1ew%2FimRNybM%3Dr8fVGMeKRaN7lrLP7noJvYUiIO7r9uIApLxiNfwziz9Fvdvtns"

successfully read config file


In [11]:
# authenticaion
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

keywords =['2023', '#python']

In [12]:
class Listener(tweepy.StreamingClient):
    
    # def __init__(self, tweet_limit = 10):
    #     self.tweet_limit = tweet_limit
    #     self.tweets = []
    #     self.tweet_counter = 1

    tweets = []
    limit = 5
    tweet_counter = 1

    def on_connect(self):
        print('connected')
        return super().on_connect()
    
    def on_tweet(self, tweet):

        if tweet.referenced_tweets == None:
            print(str(self.tweet_counter) +'/' + str(self.limit) + ' tweets \n')
            print(tweet.text + '\n')
            self.tweets.append(tweet)
            print("\033c", end="")
            self.tweet_counter += 1
            # self.tweets.append(tweet.text)
            if self.tweet_counter >= self.limit:
                self.disconnect()
                print('disconnected')

                print(len(self.tweets))
                return False

        time.sleep(0.2)

        return super().on_tweet(tweet)


    def on_status(self, status):
        self.tweets.append(status)

        if self.tweet_counter >= self.limit:
            self.disconnect()
            print('disconnected')
            return False

In [14]:
stream_tweet    = Listener(bearer_token=bearer_token)
for term in keywords:
    stream_tweet.add_rules(tweepy.StreamRule(term))

stream_tweet.filter(tweet_fields=['referenced_tweets'])

connected
1/5 tweets 

2023 people are smuggling eggs to the US… lol

c2/5 tweets 

https://t.co/G0SUvawhkB

c3/5 tweets 

GlobalData Forecasts 2.7% Growth for Nigeria’s Economy in 2023 https://t.co/SnbKkLYp7W

c

Stream connection closed by Twitter


4/5 tweets 

En #CoteIvoire, l’usine d’#aliments pour #volaille #Koudjis, filiale du #néerlandais @De_Heus_Voeders est opérationnelle avec une capacité de 120 000 tonnes https://t.co/2DdedE1fY5

cdisconnected
4


In [15]:
# create a dataframe
columns = ['ID', 'Tweet']
data = []

In [20]:

for tweet in stream_tweet.tweets:
    print(dir(tweet))
    # print(tweet.author_id)
    # data.append([tweet.user.screen_name, tweet.text])

['__abstractmethods__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '_abc_impl', 'attachments', 'author_id', 'context_annotations', 'conversation_id', 'created_at', 'data', 'edit_controls', 'edit_history_tweet_ids', 'entities', 'geo', 'get', 'id', 'in_reply_to_user_id', 'items', 'keys', 'lang', 'non_public_metrics', 'organic_metrics', 'possibly_sensitive', 'promoted_metrics', 'public_metrics', 'referenced_tweets', 'reply_settings', 'source', 'text', 'values', 'withheld']
['__abstractmethods__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribut

In [22]:
for tweet in stream_tweet.tweets:
    print(tweet.values)

<bound method Mapping.values of <Tweet id=1616487103356862474 text='2023 people are smuggling eggs to the US… lol'>>
<bound method Mapping.values of <Tweet id=1616487100563750914 text='https://t.co/G0SUvawhkB'>>
<bound method Mapping.values of <Tweet id=1616487102673190932 text='GlobalData Forecasts 2.7% Growth for Nigeria’s Economy in\xa02023 https://t.co/SnbKkLYp7W'>>
<bound method Mapping.values of <Tweet id=1616487103042289666 text='En #CoteIvoire, l’usine d’#aliments pour #volaille #Koudjis, filiale du #néerlandais @De_Heus_Voeders est opérationnelle avec une capacité de 120 000 tonnes https://t.co/2DdedE1fY5'>>


In [23]:
for tweet in stream_tweet.tweets:
    print(tweet.withheld)

None
None
None
None


In [24]:
for tweet in stream_tweet.tweets:
    
    data.append([tweet.id, tweet.text])

In [ ]:
df = pd.DataFrame(data, columns=columns)

df.to_csv('live_tweets.csv', index=False)